In [1]:
from postgres_connect import *

In [2]:
path = 'tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)

Tunnel Started
Connection Made


In [3]:
query_path = 'rider_pip_closure.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
cursor = conn.cursor()
cursor.execute(custom_query)
conn.commit()
# conn.close()

In [4]:
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)

In [5]:
df

,key,pip_date,rider_id,shipping_city,rider_name,hub,vendor_name,rider_age,need_improvement,starting_fasr,closing_fasr,pip_status,pip_result,update_time
0,20240219-Ajay_VLS_5,2024-02-19,Ajay_VLS_5,Delhi,Ajay VLS,DS DEL DWK,Vlead,146,Morning COD(58%),81.0,NaN,closed,failed,2024-02-27 18:29:31.251905
1,20240219-Swpanil_7039880610_03,2024-02-19,Swpanil_7039880610_03,Mumbai,Swpanil Khandekar,DS BOM KRL,Team HR,49,Morning Prepaid (78%),70.0,75.0,closed,passed,2024-02-27 18:29:31.251905
2,20240219-Taheer_9653638802_03,2024-02-19,Taheer_9653638802_03,Mumbai,Taheer Sharukh Shaikh,DS BOM THAN,Team HR,39,Productivity (8),75.0,87.0,closed,passed,2024-02-27 18:29:31.251905
3,20240219-Paltya_9602569772_01,2024-02-19,Paltya_9602569772_01,Bangalore,Paltya Kumar,DS BLR MTH,Peel Works,24,Productivity (13),79.0,43.0,closed,failed,2024-02-27 18:29:31.251905
4,20240219-Yahiya_7406458114_02,2024-02-19,Yahiya_7406458114_02,Bangalore,Yahiya,DS BLR BOMM,Blitz_Self,20,Productivity (15),77.0,90.0,closed,passed,2024-02-27 18:29:31.251905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,20240219-Shyam_Sunder_Porter__5,2024-02-19,Shyam_Sunder_Porter__5,Delhi,Shyam Sunder Porter,DS DEL DWK,PS Manpower,86,Morning Prepaid (67%) Productivity (13),66.0,NaN,closed,failed,2024-02-27 18:29:31.251905
72,20240219-Sumit_7011889185_024,2024-02-19,Sumit_7011889185_024,Delhi,Sumit kumar 2,DS DEL HKMN,PS Manpower Solution LLP,37,Morning COD(50%) Productivity (12),80.0,83.0,closed,failed,2024-02-27 18:29:31.251905
73,20240219-Sumit_7427016287_023,2024-02-19,Sumit_7427016287_023,Jaipur,Sumit Sain,DS JPR NPRG,JPS Group of Management,38,Morning COD(53%),64.0,81.0,closed,passed,2024-02-27 18:29:31.251905
74,20240219-Sumit_9354909075_024,2024-02-19,Sumit_9354909075_024,Delhi,Sumit Ram,DS DEL HKMN,PS Manpower Solution LLP,50,Morning COD(38%),80.0,87.0,closed,passed,2024-02-27 18:29:31.251905


In [6]:
query_path = 'rider_pip_closure_performance.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
df_pip = pd.read_sql(custom_query,conn)
df_pip.head()

,pip_date,shipping_city,rider_id,rider_name,hub,performance_type,threshold,monday,tuesday,wednesday,thursday,friday,saturday,sunday,avg_fasr,status
0,2024-02-19,Bangalore,Abhishek_MS_2,Abhishek MS,DS BLR BOMM,Morning-Prepaid,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail
1,2024-02-19,Bangalore,Abhishek_MS_2,Abhishek MS,DS BLR BOMM,Morning-COD,0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail
2,2024-02-19,Bangalore,Abhishek_MS_2,Abhishek MS,DS BLR BOMM,Evening-Prepaid,0.80,NaN,1.00,1.0,1.0,0.83,0.71,1.00,0.91,pass
3,2024-02-19,Bangalore,Abhishek_MS_2,Abhishek MS,DS BLR BOMM,Evening-COD,0.50,NaN,0.75,1.0,1.0,0.00,0.75,0.75,0.72,pass
4,2024-02-19,Bangalore,Ananda_8660619030_02,Ananda M,DS BLR BOMM,Morning-Prepaid,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail


In [7]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Filter failed riders
failed_riders = df[df['pip_result'] == 'failed']

# Group by city and hub
grouped_data = failed_riders.groupby(['shipping_city', 'hub']).size().reset_index(name='failed_count')

# Group by city to get total rider count
total_riders = df.groupby(['shipping_city', 'hub']).size().reset_index(name='total_count')

# Merge the two dataframes on city
result_df = pd.merge(total_riders, grouped_data, on=['shipping_city','hub'], how='left')

# Calculate the failed rate without decimals
result_df['failed_rate'] = (result_df['failed_count'] / result_df['total_count']) * 100
result_df['failed_rate'] = result_df['failed_rate'].fillna(0)
result_df['failed_rate'] = result_df['failed_rate'].astype(int)

# Email configuration
sender_email = "akshay@blitznow.in"
sender_password = "Outofblitz1!"

# List of recipients
recipients = [
    "operations@blitznow.in",
    "mayank@blitznow.in"
]

# Create the message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = ", ".join(recipients)
message['Subject'] = f"Rider PIP Closure Report Week - {failed_riders['pip_date'].max()}"

# Create an Excel file with two sheets
excel_file_path = 'failed_riders_report.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Write the failed_riders DataFrame to the first sheet
    failed_riders.to_excel(writer, sheet_name='Failed_Riders_Data', index=False)

    # Write the df_pip DataFrame to the second sheet
    df_pip.to_excel(writer, sheet_name='Additional_Sheet', index=False)

# Attach the Excel file
with open(excel_file_path, 'rb') as excel_file:
    attachment = MIMEApplication(excel_file.read(), _subtype='xlsx')
    attachment.add_header('Content-Disposition', 'attachment', filename='pip_closure_report.xlsx')
    message.attach(attachment)

# Create an HTML table for the email body with formatted headers
html_table = result_df.to_html(index=False, border=1, classes='table', escape=False, justify='center')
html_table = html_table.replace('<th>', '<th style="font-weight:bold; background-color:lightblue; text-align:center;">').replace('_', ' ').upper()

# Add the table to the email body
body = f"""\
<html>
  <head></head>
  <body>
    <p>Dear recipients,</p>
    <p>Please find attached the report for RIDER PIP Result for the week - {failed_riders['pip_date'].max()}.</p>
    <p>Summary:</p>
    {html_table}
    <p>Best regards,<br>Your Name</p>
  </body>
</html>
"""

message.attach(MIMEText(body, 'html'))

# Connect to the SMTP server and send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipients, message.as_string())

print("Email sent successfully.")

Email sent successfully.


In [8]:
conn.close()